In [1]:
# Import necessary libraries and functions from your IrisLocalization.py
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from IrisLocalization import *

class IrisDetectionResults:
    def __init__(self):
        self.failed_detections = []
        self.successful_detections = []
        self.processing_errors = []
        self.detection_stats = {
            'total_processed': 0,
            'successful': 0,
            'failed': 0,
            'errors': 0
        }
    
    def add_failed_detection(self, img_path, pupil_info):
        self.failed_detections.append({
            'image_path': str(img_path),
            'pupil_center': pupil_info[0],
            'pupil_radius': pupil_info[1]
        })
    
    def add_successful_detection(self, img_path, pupil_info, iris_info):
        self.successful_detections.append({
            'image_path': str(img_path),
            'pupil_center': pupil_info[0],
            'pupil_radius': pupil_info[1],
            'iris_center': (iris_info[0], iris_info[1]),
            'iris_radius': iris_info[2]
        })
    
    def add_error(self, img_path, error_msg):
        self.processing_errors.append({
            'image_path': str(img_path),
            'error': str(error_msg)
        })
    
    def update_stats(self):
        self.detection_stats['total_processed'] = len(self.successful_detections) + len(self.failed_detections) + len(self.processing_errors)
        self.detection_stats['successful'] = len(self.successful_detections)
        self.detection_stats['failed'] = len(self.failed_detections)
        self.detection_stats['errors'] = len(self.processing_errors)
    
    def generate_summary(self):
        self.update_stats()
        stats = self.detection_stats
        
        summary = f"""
Detection Summary:
-----------------
Total images processed: {stats['total_processed']}
Successful detections: {stats['successful']} ({(stats['successful']/stats['total_processed']*100):.1f}%)
Failed detections: {stats['failed']} ({(stats['failed']/stats['total_processed']*100):.1f}%)
Processing errors: {stats['errors']} ({(stats['errors']/stats['total_processed']*100):.1f}%)
        """
        return summary
    
    def save_results(self, output_dir):
        """Save detection results to files"""
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        
        # Save failed detections
        with open(output_path / 'failed_detections.txt', 'w') as f:
            for fail in self.failed_detections:
                f.write(f"Image: {fail['image_path']}\n")
                f.write(f"Pupil Center: {fail['pupil_center']}, Radius: {fail['pupil_radius']}\n\n")
        
        # Save successful detections
        with open(output_path / 'successful_detections.txt', 'w') as f:
            for success in self.successful_detections:
                f.write(f"Image: {success['image_path']}\n")
                f.write(f"Pupil Center: {success['pupil_center']}, Radius: {success['pupil_radius']}\n")
                f.write(f"Iris Center: {success['iris_center']}, Radius: {success['iris_radius']}\n\n")
        
        # Save errors
        with open(output_path / 'processing_errors.txt', 'w') as f:
            for error in self.processing_errors:
                f.write(f"Image: {error['image_path']}\n")
                f.write(f"Error: {error['error']}\n\n")
        
        # Save summary
        with open(output_path / 'summary.txt', 'w') as f:
            f.write(self.generate_summary())

def process_and_visualize_casia(base_path, output_dir='detection_results'):
    """
    Process and visualize all images in the CASIA dataset with enhanced tracking
    
    Parameters:
    base_path (str): Path to CASIA dataset root folder
    output_dir (str): Directory to save detection results
    """
    base = Path(base_path)
    results = IrisDetectionResults()
    
    try:
        # Iterate through all subjects (001-108)
        for subject in tqdm(range(1, 109), desc="Processing subjects"):
            subject_dir = f"{subject:03d}"
            
            # Process both training (1) and testing (2) directories
            for subset in ['1', '2']:
                current_dir = base / subject_dir / subset
                
                # Skip if directory doesn't exist
                if not current_dir.exists():
                    continue
                
                # Process all .bmp files in the directory
                for img_file in current_dir.glob('*.bmp'):
                    try:
                        relative_path = img_file.relative_to(base)
                        print(f"\nProcessing: {relative_path}")
                        
                        # Call your detection function
                        pupil_center, pupil_radius, iris = detect_iris_and_pupil_test(str(img_file))
                        
                        # Record results
                        if iris is not None:
                            results.add_successful_detection(
                                relative_path,
                                (pupil_center, pupil_radius),
                                iris
                            )
                            print(f"Successfully detected iris: center=({iris[0]}, {iris[1]}), radius={iris[2]}")
                        else:
                            results.add_failed_detection(
                                relative_path,
                                (pupil_center, pupil_radius)
                            )
                            print("Iris detection failed")
                            
                    except Exception as e:
                        results.add_error(relative_path, str(e))
                        print(f"Error processing {img_file.name}: {str(e)}")
        
        # Save all results
        #results.save_results(output_dir)
        
        # Print summary
        print("\n" + results.generate_summary())
        
        # Return the results object for further analysis if needed
        return results
    
    except Exception as e:
        print(f"Critical error during processing: {str(e)}")
        return results

# Example usage:
base_path = r"C:\Users\chris\OneDrive\Desktop\24fall\5293\Assignments_export\GroupProject\datasets\CASIA Iris Image Database (version 1.0)"
results = process_and_visualize_casia(base_path)

Processing subjects:   0%|          | 0/108 [00:00<?, ?it/s]


Processing: 001\1\001_1_1.bmp

Selected circle metrics:
Final score: 0.724
Radius score: 0.832 (radius: 103)
Center score: 0.723 (distance: 6.3)
Successfully detected iris: center=(176, 131), radius=103

Processing: 001\1\001_1_2.bmp

Selected circle metrics:
Final score: 0.814
Radius score: 0.895 (radius: 100)
Center score: 0.814 (distance: 4.5)
Successfully detected iris: center=(170, 139), radius=100

Processing: 001\1\001_1_3.bmp

Selected circle metrics:
Final score: 0.737
Radius score: 0.832 (radius: 103)
Center score: 0.736 (distance: 6.3)
Successfully detected iris: center=(168, 118), radius=103

Processing: 001\2\001_2_1.bmp

Selected circle metrics:
Final score: 0.771
Radius score: 0.895 (radius: 100)
Center score: 0.770 (distance: 5.7)
Successfully detected iris: center=(179, 118), radius=100

Processing: 001\2\001_2_2.bmp

Selected circle metrics:
Final score: 0.809
Radius score: 0.789 (radius: 105)
Center score: 0.809 (distance: 4.5)
Successfully detected iris: center=(17

Processing subjects:   1%|          | 1/108 [00:02<04:43,  2.65s/it]

Successfully detected iris: center=(149, 132), radius=103

Processing: 002\1\002_1_1.bmp

No iris detected. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(180, 140), radius=108

Processing: 002\1\002_1_2.bmp

Selected circle metrics:
Final score: 0.472
Radius score: 0.768 (radius: 106)
Center score: 0.469 (distance: 15.6)
Best detection score (0.472) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(183, 138), radius=106

Processing: 002\1\002_1_3.bmp

Selected circle metrics:
Final score: 0.364
Radius score: 0.768 (radius: 106)
Center score: 0.360 (distance: 18.4)
Best detection score (0.364) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(171, 155), radius=105

Processing: 002\2\002_2_1.bmp

Selected circle metrics:
Final score: 0.420
Radius score: 0.874 (radius:

Processing subjects:   2%|▏         | 2/108 [00:05<04:40,  2.64s/it]

...

Successfully detected iris: center=(147, 134), radius=109

Processing: 107\1\107_1_1.bmp

Selected circle metrics:
Final score: 0.850
Radius score: 0.726 (radius: 108)
Center score: 0.851 (distance: 4.5)
Successfully detected iris: center=(147, 151), radius=108

Processing: 107\1\107_1_2.bmp

Selected circle metrics:
Final score: 0.737
Radius score: 0.916 (radius: 99)
Center score: 0.736 (distance: 8.2)
Successfully detected iris: center=(161, 137), radius=99

Processing: 107\1\107_1_3.bmp

Selected circle metrics:
Final score: 0.576
Radius score: 0.768 (radius: 106)
Center score: 0.574 (distance: 12.0)
Best detection score (0.576) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(157, 125), radius=104

Processing: 107\2\107_2_1.bmp

Selected circle metrics:
Final score: 0.863
Radius score: 0.726 (radius: 108)
Center score: 0.864 (distance: 4.0)
Successfully detected iris: center=(166, 143), r

Processing subjects:  99%|█████████▉| 107/108 [06:07<00:05,  5.24s/it]

Successfully detected iris: center=(178, 131), radius=97

Processing: 108\1\108_1_1.bmp

Selected circle metrics:
Final score: 0.764
Radius score: 0.768 (radius: 106)
Center score: 0.764 (distance: 5.7)
Successfully detected iris: center=(157, 158), radius=106

Processing: 108\1\108_1_2.bmp

Selected circle metrics:
Final score: 0.681
Radius score: 0.768 (radius: 106)
Center score: 0.680 (distance: 8.2)
Successfully detected iris: center=(170, 159), radius=106

Processing: 108\1\108_1_3.bmp

Selected circle metrics:
Final score: 0.205
Radius score: 0.726 (radius: 82)
Center score: 0.200 (distance: 22.1)
Best detection score (0.205) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(163, 140), radius=103

Processing: 108\2\108_2_1.bmp

No iris detected. Using fallback mechanism (pupil_radius + 57).
Successfully detected iris: center=(169, 146), radius=101

Processing: 108\2\108_2_2.bmp

Selected

Processing subjects: 100%|██████████| 108/108 [06:10<00:00,  3.43s/it]

Successfully detected iris: center=(168, 155), radius=99


Detection Summary:
-----------------
Total images processed: 756
Successful detections: 756 (100.0%)
Failed detections: 0 (0.0%)
Processing errors: 0 (0.0%)
        


In [1]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
from IrisLocalization import detect_iris_and_pupil
from IrisNormalization import normalize_iris
from IrisEnhancement import enhance_iris

def normalize_enhance_all_iris(base_path, output_dir='detection_results'):
    """
    Process all images in the dataset, applying iris detection, normalization at multiple angles, 
    and enhancement, and save the processed images with angle-specific filenames.
    
    Parameters:
        base_path (str): Path to the CASIA dataset root folder.
        output_dir (str): Directory to save the detection results.
    """
    base = Path(base_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    # Iterate through all subjects (001-108)
    for subject in tqdm(range(1, 109), desc="Processing subjects"):
        subject_dir = f"{subject:03d}"
        
        # Process both training (1) and testing (2) directories
        for subset in ['1', '2']:
            current_dir = base / subject_dir / subset
            
            # Skip if directory doesn't exist
            if not current_dir.exists():
                continue
            
            # Process all .bmp files in the directory
            for img_file in current_dir.glob('*.bmp'):
                try:
                    # Load the image
                    image = cv2.imread(str(img_file), cv2.IMREAD_GRAYSCALE)
                    
                    # Perform detection of iris and pupil
                    final_pupil_center, pupil_radius, iris_circle = detect_iris_and_pupil(str(img_file))
                    
                    # Check if both pupil and iris were detected
                    if final_pupil_center and iris_circle:
                        for angle in [-9, -6, -3, 0, 3, 6, 9]:
                            # Normalize the iris with the current angle
                            normalized_iris = normalize_iris(image, final_pupil_center, pupil_radius, iris_circle, initial_angle=angle)
                            
                            # Enhance the normalized iris image
                            enhanced_iris = enhance_iris(normalized_iris)
                            
                            # Create a filename for the enhanced image
                            output_filename = f"{img_file.stem}_angle_{angle}.png"
                            output_path = output_dir / output_filename
                            
                            # Save the enhanced normalized image
                            cv2.imwrite(str(output_path), enhanced_iris)
                    
                except Exception as e:
                    print(f"Error processing {img_file}: {e}")

# Example of calling the function
base_path = r"C:\Users\chris\OneDrive\Desktop\24fall\5293\Assignments_export\GroupProject\datasets\CASIA Iris Image Database (version 1.0)"
normalize_enhance_all_iris(base_path, output_dir=r"C:\Users\chris\OneDrive\Desktop\24fall\5293\Assignments_export\GroupProject\datasets\Enhanced_Normalized_Iris_7angle")


Processing subjects:   0%|          | 0/108 [00:00<?, ?it/s]


Selected circle metrics:
Final score: 0.724
Radius score: 0.832 (radius: 103)
Center score: 0.723 (distance: 6.3)

Selected circle metrics:
Final score: 0.814
Radius score: 0.895 (radius: 100)
Center score: 0.814 (distance: 4.5)

Selected circle metrics:
Final score: 0.737
Radius score: 0.832 (radius: 103)
Center score: 0.736 (distance: 6.3)

Selected circle metrics:
Final score: 0.771
Radius score: 0.895 (radius: 100)
Center score: 0.770 (distance: 5.7)

Selected circle metrics:
Final score: 0.809
Radius score: 0.789 (radius: 105)
Center score: 0.809 (distance: 4.5)

Selected circle metrics:
Final score: 0.813
Radius score: 0.789 (radius: 105)
Center score: 0.814 (distance: 4.5)


Processing subjects:   1%|          | 1/108 [00:01<02:15,  1.26s/it]


Selected circle metrics:
Final score: 0.799
Radius score: 0.832 (radius: 103)
Center score: 0.799 (distance: 4.5)

No iris detected. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.472
Radius score: 0.768 (radius: 106)
Center score: 0.469 (distance: 15.6)
Best detection score (0.472) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.364
Radius score: 0.768 (radius: 106)
Center score: 0.360 (distance: 18.4)
Best detection score (0.364) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.420
Radius score: 0.874 (radius: 101)
Center score: 0.415 (distance: 17.9)
Best detection score (0.420) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

No iris detected. Using fallback mechanism (pupil_radius + 5

Processing subjects:   2%|▏         | 2/108 [00:02<02:15,  1.28s/it]

...

Processing subjects:  98%|█████████▊| 106/108 [03:19<00:04,  2.22s/it]


Selected circle metrics:
Final score: 0.850
Radius score: 0.726 (radius: 108)
Center score: 0.851 (distance: 4.5)

Selected circle metrics:
Final score: 0.737
Radius score: 0.916 (radius: 99)
Center score: 0.736 (distance: 8.2)

Selected circle metrics:
Final score: 0.576
Radius score: 0.768 (radius: 106)
Center score: 0.574 (distance: 12.0)
Best detection score (0.576) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.863
Radius score: 0.726 (radius: 108)
Center score: 0.864 (distance: 4.0)

Selected circle metrics:
Final score: 0.243
Radius score: 0.726 (radius: 82)
Center score: 0.238 (distance: 25.6)
Best detection score (0.243) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.137
Radius score: 0.811 (radius: 86)
Center score: 0.131 (distance: 29.7)
Best detection score (0.137) below t

Processing subjects:  99%|█████████▉| 107/108 [03:21<00:02,  2.19s/it]


Selected circle metrics:
Final score: 0.764
Radius score: 0.768 (radius: 106)
Center score: 0.764 (distance: 5.7)

Selected circle metrics:
Final score: 0.681
Radius score: 0.768 (radius: 106)
Center score: 0.680 (distance: 8.2)

Selected circle metrics:
Final score: 0.205
Radius score: 0.726 (radius: 82)
Center score: 0.200 (distance: 22.1)
Best detection score (0.205) below threshold (0.62), using fallback.

Low detection score. Using fallback mechanism (pupil_radius + 57).

No iris detected. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.746
Radius score: 0.958 (radius: 97)
Center score: 0.744 (distance: 7.2)

No iris detected. Using fallback mechanism (pupil_radius + 57).

Selected circle metrics:
Final score: 0.603
Radius score: 0.600 (radius: 114)
Center score: 0.603 (distance: 10.0)
Best detection score (0.603) below threshold (0.62), using fallback.
Detected radius (114) exceeds maximum (110), using fallback.

Low detection score. Using 

Processing subjects: 100%|██████████| 108/108 [03:23<00:00,  1.88s/it]
